# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [12]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [13]:
conn = psycopg2.connect("host=127.0.0.1 port=5432 dbname=postgres user=postgres password=udacitytest")
cur = conn.cursor()

In [14]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
#song_files = 'TRAABJL12903CDCF1A.json'
song_files = get_files(filepath='data/song_data')

In [5]:
filepath = song_files[0]
#filepath = os.path.join('data/song_data/A/A/B/',song_files)

In [6]:
df = pd.read_json(filepath, lines=True)  # 'data/song_data/A/A/B/TRAABJL12903CDCF1A.json'
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
df['year'] = df['year'].astype(float)
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
# other possiblity
# song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].iloc[0].values.tolist()
song_data

['SOMZWCG12A8C13C480',
 "I Didn't Mean To",
 'ARD7TVE1187B99BFB1',
 0.0,
 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
for i in song_data:
    cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
for i in artist_data:   
    cur.execute(artist_table_insert, artist_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files(filepath="data/log_data")
#get_files('FirstProject/data/logdata/2018/11/')

In [17]:
filepath = log_files[0]

In [18]:
df = pd.read_json(filepath, lines=True)
#df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = pd.read_json(filepath, lines=True)
filterdata = ["NextSong"]
df = df[df.page.isin(filterdata)]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [20]:
t = pd.to_datetime(df['ts'])
t.head()


2   1970-01-01 00:25:41.106106796
4   1970-01-01 00:25:41.106352796
5   1970-01-01 00:25:41.106496796
6   1970-01-01 00:25:41.106673796
7   1970-01-01 00:25:41.107053796
Name: ts, dtype: datetime64[ns]

In [21]:
timedata = [t, t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month, t.dt.year, t.dt.weekday]
timedata

[2    1970-01-01 00:25:41.106106796
 4    1970-01-01 00:25:41.106352796
 5    1970-01-01 00:25:41.106496796
 6    1970-01-01 00:25:41.106673796
 7    1970-01-01 00:25:41.107053796
 8    1970-01-01 00:25:41.107493796
 9    1970-01-01 00:25:41.107734796
 10   1970-01-01 00:25:41.108520796
 12   1970-01-01 00:25:41.109125796
 13   1970-01-01 00:25:41.109325796
 14   1970-01-01 00:25:41.110994796
 Name: ts, dtype: datetime64[ns],
 2     0
 4     0
 5     0
 6     0
 7     0
 8     0
 9     0
 10    0
 12    0
 13    0
 14    0
 Name: ts, dtype: int64,
 2     1
 4     1
 5     1
 6     1
 7     1
 8     1
 9     1
 10    1
 12    1
 13    1
 14    1
 Name: ts, dtype: int64,
 2     1
 4     1
 5     1
 6     1
 7     1
 8     1
 9     1
 10    1
 12    1
 13    1
 14    1
 Name: week, dtype: UInt32,
 2     1
 4     1
 5     1
 6     1
 7     1
 8     1
 9     1
 10    1
 12    1
 13    1
 14    1
 Name: ts, dtype: int64,
 2     1970
 4     1970
 5     1970
 6     1970
 7     1970
 8     1970

In [22]:
#time_data = ()
column_labels = ['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday']
test = dict(zip(column_labels, timedata))
time_df = pd.DataFrame.from_dict(test)
time_df.head()

,timestamp,hour,day,week,month,year,weekday
2,1970-01-01 00:25:41.106106796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.106352796,0,1,1,1,1970,3
5,1970-01-01 00:25:41.106496796,0,1,1,1,1970,3
6,1970-01-01 00:25:41.106673796,0,1,1,1,1970,3
7,1970-01-01 00:25:41.107053796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [11]:
import io
def copy_from_stringio(conn, time_df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = io.StringIO()
    time_df.to_csv(buffer, index = False, header=False)
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()
    
#-----------------------------------------------
# Main code
#-----------------------------------------------
conn = psycopg2.connect("host=127.0.0.1 port=5432 dbname=postgres user=postgres password=udacitytest") # connect to the database
copy_from_stringio(conn, time_df, 'time') # copy the dataframe to SQL
#conn.close() # close the connection

copy_from_stringio() done


In [23]:
import io
def copy_from_stringio(conn, time_df, table):
    buffer = io.StringIO()
    time_df.to_csv(buffer, index= False, header=False)
    buffer.seek(0)
    cursor = conn.cursor()
    #print(buffer.read())
    cursor.copy_from(buffer, table, sep=",")
    conn.commit()
    cursor.close()
    
copy_from_stringio(conn, time_df, 'time')

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
#select = ['userId', 'firstName', 'lastName', 'gender', 'level']
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
# hier muss ich noch etwas mergen
songplay_data = df[['userId', 'level', 'sessionId', 'location', 'userAgent']]
songplay_data.head()

In [22]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    # insert songplay record
songplay_data = (pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
cur.execute(songplay_table_insert, songplay_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [23]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.